## N번 촬영한 마커 이미지를 평균

In [33]:
import os
import json
import cv2
import numpy as np
from collections import defaultdict
from scipy.spatial.transform import Rotation as R

# 디렉토리 설정
base_dirs = [
    "./1_ArUco_cap",
    "./2_ArUco_cap",
    "./3_ArUco_cap"
]
output_dir = "./Correct_ArUco"
os.makedirs(output_dir, exist_ok=True)

# 파일 이름에서 뷰와 카메라 추출
def parse_filename(filename):
    parts = filename.split('_')
    view = parts[0]
    cam = parts[2]
    return view, cam

# 데이터 구조: data[view][cam][marker_id] = list of marker dicts
data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

# 파일 수집 및 데이터 적재
for base_dir in base_dirs:
    for fname in os.listdir(base_dir):
        if not fname.endswith('.json'):
            continue
        view, cam = parse_filename(fname)
        with open(os.path.join(base_dir, fname), 'r') as f:
            content = json.load(f)
            for marker_id, marker_data in content.items():
                data[view][cam][marker_id].append(marker_data)

# 회전 평균
def average_quaternion(quaternions):
    r = R.from_quat(quaternions)
    return R.mean(r).as_quat()

# 위치 평균
def average_position(positions):
    return np.mean(positions, axis=0)

# 마커 정리 및 저장
for view in data: # ['top', 'right', 'front', 'left']
    for cam in data[view]: # ['leftcam', 'rightcam']
        corrected = {}
        for marker_ids, entries in data[view][cam].items():
            if len(entries) < 2:
                continue  # 최소 두 개 이상 있어야 평균
            positions = [np.array([m['position_m']['x'], m['position_m']['y'], m['position_m']['z']]) for m in entries]
            quaternions = [np.array([m['rotation_quat']['x'], m['rotation_quat']['y'],
                                    m['rotation_quat']['z'], m['rotation_quat']['w']]) for m in entries]

            avg_pos = average_position(positions)
            avg_quat = average_quaternion(quaternions)

            corrected[marker_ids] = {
                "position_m": {"x": float(avg_pos[0]), "y": float(avg_pos[1]), "z": float(avg_pos[2])},
                "rotation_quat": {"x": float(avg_quat[0]), "y": float(avg_quat[1]),
                                "z": float(avg_quat[2]), "w": float(avg_quat[3])},
                "corners_pixel": entries[0]["corners_pixel"]
            }

        output_path = os.path.join(output_dir, f"{view}_{cam}_corrected.json")
        with open(output_path, 'w') as f:
            json.dump(corrected, f, indent=4)

print("보정된 결과가 다음 디렉토리에 저장되었습니다:", output_dir)

보정된 결과가 다음 디렉토리에 저장되었습니다: ./Correct_ArUco


## 평균값에서 회전은 동일하다고 가정하여, 거리별 가중치 노이즈 제거하여 회전 벡터 구하기

In [48]:
# 디렉토리 설정
corrected_dirs = "./Correct_ArUco"
output_dir = "./Rot_Tvec_corrected_ArUco"
os.makedirs(output_dir, exist_ok=True)

# 파일 이름에서 뷰와 카메라 추출
def parse_filename(filename):
    parts = filename.split('_')
    view = parts[0]
    cam = parts[1]
    return view, cam

# 데이터 구조: data[view][cam][marker_id] = list of marker dicts
data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

# 파일 수집 및 데이터 적재
for fname in os.listdir(corrected_dirs):
    if not fname.endswith('.json'):
        continue
    view, cam = parse_filename(fname)
    with open(os.path.join(corrected_dirs, fname), 'r') as f:
        content = json.load(f)
        for marker_id, marker_data in content.items():
            data[view][cam][marker_id].append(marker_data)


ANGLE_THRESH_DEG = 1.0  # 회전 이상치 판정 임계각

for view, cams in data.items():
    for cam, markers in cams.items():
        # (1) 포즈 수집
        tvecs, quats, ids = [], [], []
        for marker_id, entries in markers.items():
            m = entries[0]
            tvecs.append([m['position_m'][k] for k in ('x','y','z')])
            quats.append([m['rotation_quat'][k] for k in ('x','y','z','w')])
            ids.append(marker_id)
        tvecs = np.array(tvecs); quats = np.array(quats)
        N = len(ids)
        if N < 2:
            print(f"{view}/{cam}: 마커 {N}개라 스킵")
            continue

        # (2) 거리 가중치
        dists = np.linalg.norm(tvecs, axis=1)
        weights = 1.0/(dists+1e-6)
        weights /= weights.sum()

        # (3) 회전 이상치 제거
        def total_angdist(qi, all_q):
            dots = np.abs(np.dot(all_q, qi))
            return np.sum(2*np.arccos(np.clip(dots, -1,1)))
        scores = [ total_angdist(quats[i], quats) for i in range(N) ]
        q_med = quats[np.argmin(scores)]
        dots = np.abs(np.dot(quats, q_med))
        thetas = 2*np.arccos(np.clip(dots, -1,1))
        mask = thetas < np.deg2rad(ANGLE_THRESH_DEG)

        excluded = [ids[i] for i in range(N) if not mask[i]]
        if excluded:
            print(f"[{view}/{cam}] 제외된 마커 (θ≥{ANGLE_THRESH_DEG}°): {excluded}")
        if mask.sum() < 2:
            print(f"{view}/{cam}: inlier <2 → 스킵")
            continue

        # (4) 가중 회전 평균
        quats_f = quats[mask]
        w_f = weights[mask]; w_f /= w_f.sum()
        M = sum(w * np.outer(q, q) for q, w in zip(quats_f, w_f))
        eigvals, eigvecs = np.linalg.eigh(M)
        q_mean = eigvecs[:, np.argmax(eigvals)]; q_mean /= np.linalg.norm(q_mean)

        # (5) tvec 보정
        R_mean = R.from_quat(q_mean).as_matrix()
        tvecs_new = {}
        for idx, mid in enumerate(ids):
            R_i = R.from_quat(quats[idx]).as_matrix()
            R_diff = R_mean @ R_i.T
            t_orig = tvecs[idx].reshape(3,1)
            t_corr = (R_diff @ t_orig).flatten().tolist()
            tvecs_new[mid] = t_corr

        # (6) 출력 JSON 생성 & 저장
        corrected_output = {}
        for mid in ids:
            corrected_output[mid] = {
                "position_m": {
                    "x": tvecs_new[mid][0],
                    "y": tvecs_new[mid][1],
                    "z": tvecs_new[mid][2]
                },
                "rotation_quat": {
                    "x": float(q_mean[0]),
                    "y": float(q_mean[1]),
                    "z": float(q_mean[2]),
                    "w": float(q_mean[3])
                },
                "corners_pixel": markers[mid][0]["corners_pixel"]
            }

        out_fname = f"{view}_{cam}_RotTvec_corrected.json"
        with open(os.path.join(output_dir, out_fname), "w") as fout:
            json.dump(corrected_output, fout, indent=4)

        print(f"[{view}/{cam}] 저장: {out_fname}")

[left/leftcam] 제외된 마커 (θ≥1.0°): ['4']
[left/leftcam] 저장: left_leftcam_RotTvec_corrected.json
[left/rightcam] 제외된 마커 (θ≥1.0°): ['4']
[left/rightcam] 저장: left_rightcam_RotTvec_corrected.json
[top/leftcam] 제외된 마커 (θ≥1.0°): ['7', '8', '3', '5']
[top/leftcam] 저장: top_leftcam_RotTvec_corrected.json
[top/rightcam] 제외된 마커 (θ≥1.0°): ['8', '7', '5']
[top/rightcam] 저장: top_rightcam_RotTvec_corrected.json
[right/leftcam] 제외된 마커 (θ≥1.0°): ['8']
[right/leftcam] 저장: right_leftcam_RotTvec_corrected.json
[right/rightcam] 제외된 마커 (θ≥1.0°): ['8']
[right/rightcam] 저장: right_rightcam_RotTvec_corrected.json
[front/leftcam] 제외된 마커 (θ≥1.0°): ['1', '6']
[front/leftcam] 저장: front_leftcam_RotTvec_corrected.json
[front/rightcam] 제외된 마커 (θ≥1.0°): ['1', '6']
[front/rightcam] 저장: front_rightcam_RotTvec_corrected.json


## 구한 회전값을 토대로 기존 값 보정